# 1. Instalar librerias

In [ ]:
!pip install ydata-profiling

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00


# 2. Cargar librerias

In [ ]:
import pandas as pd
import chardet
from ydata_profiling import ProfileReport
import numpy as np
import csv
from io import StringIO
import os

# 3. Funciones

In [ ]:
def DetectaEncoding(url):
  #Detectar el encoding de csv
  with open(url, 'rb') as f:
    result = chardet.detect(f.read(50000))
    encoding = result['encoding']
    if encoding.lower() == 'ascii':
      #Forzamos un encoding más permisivo
      encoding = 'latin-1'
    if encoding.lower() == 'big5' :
      #Forzamos un encoding más permisivo
      encoding = 'cp1252'
    #if encoding.lower() == 'macroman':
      #encoding = 'ISO-8859-15'

  print(encoding)
  return encoding


def EDA(df, title):
  EDA = ProfileReport(df, title="EDA - " + title, explorative=True)
  EDA.to_notebook_iframe()
  #EDA_medicacion.to_file("EDA_Terrazas.html")
  #df1.to_csv('Terrazas_202104_EDA.csv', sep=";", encoding='latin-1', index=False)


def DetectaSeparador(url, num_lineas=5):
    with open(url, 'r', encoding=DetectaEncoding(url), errors='ignore') as f:
        muestra = ''.join([f.readline() for _ in range(num_lineas)])
        dialecto = csv.Sniffer().sniff(muestra)
        return dialecto.delimiter

#Corregimos los caracteres que nhace que los archivos sean reconocidos como una columna aparte
def CorreccionSaltoLinea(url):
  encoding = DetectaEncoding(url)
  separador = DetectaSeparador(url)
  lineas_corregidas = []
  with open(url, 'r', encoding=encoding) as f_in:
    for line in f_in:
      parts = line.strip().split(separador)
      if len(parts) > 8:
        # Corregir campo resultado uniendo lo intermedio y limpiando salto extraño
        resultado = separador.join(parts[5:-2]).replace('\\X0A\\', '\n')
        #resultado = ';'.join(parts[5:-2]).replace('\\X0A\\', '\n')
        linea_corregida = ';'.join(parts[:5] + [f'"{resultado}"'] + parts[-2:]) + '\n'
        lineas_corregidas.append(linea_corregida)
      else:
        lineas_corregidas.append(line.strip())
  # Convertir lista de líneas a un DataFrame
  contenido_corregido = '\n'.join(lineas_corregidas)
  df = pd.read_csv(StringIO(contenido_corregido), sep=';', encoding=encoding)
  return df

'''
def CambiarNombre(df, columnas_antiguas, columnas_nuevas):
  if len(columnas_antiguas) != len(columnas_nuevas):
    raise ValueError("Las listas deben tener la misma longitud.")

  mapeo = dict(zip(columnas_antiguas, columnas_nuevas))
  df.rename(columns=mapeo, inplace=True)
'''

def CambiarNombre(df, mapa_columnas):
  renombres = {}
  columnas_actuales = df.columns.str.upper().str.strip()  # estandariza mayúsculas y espacios
  #print(columnas_actuales)

  for nombre_estandar, posibles_nombres in mapa_columnas.items(): #recorro los nombres que le daremos desde el mapa
    #print(nombre_estandar)
    for posible in posibles_nombres: #recorro el posible nombre que se llama en el  df original desde mapa
      for col_actual in columnas_actuales: #recorro los nombres actuales del df original
        if posible.upper().strip() == col_actual: #si hay match , hay rename
          renombres[df.columns[columnas_actuales.tolist().index(col_actual)]] = nombre_estandar
          #print(f"Renombrando {posible} a {nombre_estandar}")
          continue  # Para con la primera coincidencia

  df.rename(columns=renombres, inplace=True)
  return df


# 4. Directorios

In [ ]:
#Direcorios

ruta = 'Unificados'

# verificamos si el directorio ya existe
if not os.path.exists(ruta):
  os.mkdir(ruta)
  print("Directorio %s creado!" % ruta)
else:
  print("Directorio %s ya existe" % ruta)

Directorio Unificados creado!


# 5. Cargar Archivos

## 1. Ingresos hospitalarios y urgencias.

### LOAD

#### Unificación

In [ ]:

mapa_nombres = {
                  'ID_PACIENTE': ['PATIENT ID', 'IDINGRESO'],
                  'EDAD': ['EDAD/AGE', 'EDAD'],
                  'SEXO': ['SEXO/SEX', 'SEX'],
                  'DIAGNOSTICO_INGRESO': ['DIAG ING/INPAT', 'DIAGNOSTICO_ING'],
                  'FECHA_INGRESO': ['F_INGRESO/ADMISSION_D_ING/INPAT', 'F_INGRESO_ING'],
                  'FECHA_UCI_INGRESO': ['F_ENTRADA_UC/ICU_DATE_IN', 'F_UCI_IN'],
                  'FECHA_UCI_SALIDA': ['F_SALIDA_UCI/ICU_DATE_OUT', 'F_UCI_OUT'],
                  'UCI_DIAS': ['UCI_DIAS/ICU_DAYS', 'UCI_DAYS'],
                  'FECHA_ALTA_INGRESO': ['F_ALTA/DISCHARGE_DATE_ING', 'F_ALTA_ING'],
                  'MOTIVO_ALTA_INGRESO': ['MOTIVO_ALTA/DESTINY_DISCHARGE_ING', 'MOTIVO_ALTA_ING'],
                  'FECHA_URGENCIA_INGRESO': ['F_INGRESO/ADMISSION_DATE_URG/EMERG', 'F_INGRESO_URG'],
                  'HORA_URGENCIA_INGRESO': ['HORA/TIME_ADMISION/ADMISSION_URG/EMERG', 'HORA_URG'],
                  'ESPECIALIDAD_URGENCIA': ['ESPECIALIDAD/DEPARTMENT_URG/EMERG', 'ESPECIALIDAD_URGENCIA'],
                  'DIAGNOSTICO_URGENCIA': ['DIAG_URG/EMERG', 'DIAG_URG'],
                  'DESTINO_URGENCIA': ['DESTINO/DESTINY_URG/EMERG', 'DESTINO_URG'],
                  'HORA_CONSTANTES_PRIMERA_URGENCIA': ['HORA/TIME_CONSTANT_PRIMERA/FIRST_URG/EMERG', 'HORA_CONSTANTES_PRIMERA_URG'],
                  'TEMPERATURA_PRIMERA_URGENCIA': ['TEMP_PRIMERA/FIRST_URG/EMERG', 'TEMP_PRIMERA_URG'],
                  'FC_PRIMERA_URGENCIA': ['FC/HR_PRIMERA/FIRST_URG/EMERG', 'FC_PRIMERA_URG'],
                  'GLU_PRIMERA_URGENCIA': ['GLU_PRIMERA/FIRST_URG/EMERG', 'GLU_PRIMERA_URG'],
                  'SAT_02_PRIMERA_URGENCIA': ['SAT_02_PRIMERA/FIRST_URG/EMERG', 'SAT_02_PRIMERA_URG'],
                  'TA_MAX_PRIMERA_URGENCIA': ['TA_MAX_PRIMERA/FIRST/EMERG_URG', 'TA_MAX_PRIMERA_URG'],
                  'TA_MIN_PRIMERA_URGENCIA': ['TA_MIN_PRIMERA/FIRST_URG/EMERG', 'TA_MIN_PRIMERA_URG'],
                  'HORA_CONSTANTES_ULTIMA_URGENCIA': ['HORA/TIME_CONSTANT_ULTIMA/LAST_URG/EMERG', 'HORA_CONSTANTES_ULTIMA_URG'],
                  'FC_ULTIMA_URGENCIA': ['FC/HR_ULTIMA/LAST_URG/EMERG', 'FC_ULTIMA_URG'],
                  'TEMPERATURA_ULTIMA_URGENCIA': ['TEMP_ULTIMA/LAST_URG/EMERG', 'TEMP_ULTIMA_URG'],
                  'GLU_ULTIMA_URGENCIA': ['GLU_ULTIMA/LAST_URG/EMERG', 'GLU_ULTIMA_URG'],
                  'SAT_02_ULTIMA_URGENCIA': ['SAT_02_ULTIMA/LAST_URG/EMERG', 'SAT_02_ULTIMA_URG'],
                  'TA_MAX_ULTIMA_URGENCIA': ['TA_MAX_ULTIMA/LAST_URGEMERG', 'TA_MAX_ULTIMA_URG'],
                  'TA_MIN_ULTIMA_URGENCIA': ['TA_MIN_ULTIMA/LAST_URG/EMERG', 'TA_MIN_ULTIMA_URG'],
                  'DIURESIS_PRIMERA_URGENCIA': ['DIURESIS_PRIMERA_URG'],
                  'FECHA_INGRESO_ANTERIOR': ['F_ING_ANT'],
                  'DIAGNOSTICO_ANTERIOR': ['DIAG_ANT'],
                  'RESPIRADOR': ['RESPIRADOR'],
                  'UCI_N_ING': ['UCI_N_ING']
}

#urls = {"01.csv":"F_24_04_2020","CDSL_01.csv":"F_20_07_2020","COVID_DSL_01.CSV":"F_19_04_2021"}
urls = {"01.csv":"F_24_04_2020"}

df_ingresos = pd.DataFrame()
for url,fuente in urls.items():
  df_ingresos_temp =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
  df_ingresos_temp["FUENTE"] = fuente
  CambiarNombre(df_ingresos_temp, mapa_nombres)
  df_ingresos = pd.concat([df_ingresos, df_ingresos_temp], ignore_index=True)
  print(df_ingresos_temp.shape)
print(df_ingresos.shape)
df_ingresos

MacRoman
MacRoman
(2307, 30)
(2307, 30)


,ID_PACIENTE,EDAD,SEXO,DIAGNOSTICO_INGRESO,FECHA_INGRESO,FECHA_UCI_INGRESO,FECHA_UCI_SALIDA,UCI_DIAS,FECHA_ALTA_INGRESO,MOTIVO_ALTA_INGRESO,...,TA_MAX_PRIMERA_URGENCIA,TA_MIN_PRIMERA_URGENCIA,HORA_CONSTANTES_ULTIMA_URGENCIA,FC_ULTIMA_URGENCIA,TEMPERATURA_ULTIMA_URGENCIA,GLU_ULTIMA_URGENCIA,SAT_02_ULTIMA_URGENCIA,TA_MAX_ULTIMA_URGENCIA,TA_MIN_ULTIMA_URGENCIA,FUENTE
0,577,78,MALE,COVID19 - POSITIVO,26/12/2019,26/12/2019 17:12,27/12/2019 12:22,1.0,02/04/2020,Domicilio,...,0,0,NaN,0,0,0,0,0,0,F_24_04_2020
1,44,75,FEMALE,COVID19 - POSITIVO,28/01/2020,30/01/2020 13:03,31/01/2020 17:08,1.0,04/04/2020,Domicilio,...,0,0,NaN,0,0,0,0,0,0,F_24_04_2020
2,585,62,FEMALE,COVID19 - POSITIVO,05/02/2020,10/03/2020 14:20,20/03/2020 14:11,10.0,26/03/2020,Domicilio,...,160,101,18:34:00,105,0,0,97,160,101,F_24_04_2020
3,587,69,MALE,COVID19 - POSITIVO,06/02/2020,NaN,NaN,NaN,24/03/2020,Domicilio,...,141,83,18:54:00,92,37,0,95,141,83,F_24_04_2020
4,586,67,FEMALE,COVID19 - POSITIVO,06/02/2020,NaN,NaN,NaN,28/03/2020,Fallecimiento,...,151,82,0:55:00,84,36,0,93,151,82,F_24_04_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2302,280,87,MALE,COVID19 - POSITIVO,19/04/2020,NaN,NaN,NaN,NaN,NaN,...,168,83,18:05:00,94,37,0,97,168,83,F_24_04_2020
2303,273,52,FEMALE,COVID19 - POSITIVO,19/04/2020,NaN,NaN,NaN,21/04/2020,Domicilio,...,120,73,15:11:00,87,"36,5",0,97,120,73,F_24_04_2020
2304,285,79,MALE,COVID19 - POSITIVO,20/04/2020,23/04/2020 8:30,NaN,NaN,NaN,NaN,...,119,69,23:53:00,92,"37,59999847",0,93,119,69,F_24_04_2020
2305,287,35,FEMALE,COVID19 - POSITIVO,20/04/2020,NaN,NaN,NaN,21/04/2020,Domicilio,...,0,0,7:39:00,84,"36,09999847",0,97,0,0,F_24_04_2020


#### Exportación

In [ ]:
df_ingresos.to_csv("Unificados/01_ingresos.csv", index=False, sep=";",encoding='utf-8')

### EDA

In [ ]:
EDA(df_ingresos, "Ingresos")

## 2. Medicación

### LOAD

#### Unificación

In [ ]:
mapa_nombres = {
    'ID_PACIENTE': ['PATIENT ID', 'IDINGRESO'],
    'FARMACO_NOMBRE_COMERCIAL': ['FARMACO/DRUG_NOMBRE_COMERCIAL/COMERCIAL_NAME', 'DRUG_COMERCIAL_NAME'],
    'DOSIS_MEDIA_DIARIA': ['DOSIS_MEDIA_DIARIA/DAILY_AVRG_DOSE', 'DAILY_AVRG_DOSE'],
    'INICIO_TRATAMIENTO': ['INICIO_TRAT/DRUG_START_DATE', 'DRUG_START_DATE'],
    'FIN_TRATAMIENTO': ['FIN_TRAT/DRUG_END_DATE', 'DRUG_END_DATE'],
    'ATC5_NOMBRE': ['ATC5_NOMBRE/NAME', 'ATC5_NAME'],
    'ID_ATC5': ['ID_ATC5', 'ID_ATC5'],
    'ATC7_NOMBRE': ['ATC7_NOMBRE/NAME', 'ATC7_NAME'],
    'ID_ATC7': ['ID_ATC7', 'ID_ATC7']
}


urls = {"02.csv":"F_24_04_2020","CDSL_02.csv":"F_20_07_2020","COVID_DSL_04.CSV":"F_19_04_2021"}

df_medicacion = pd.DataFrame()
for url, fuente in urls.items():
  df_medicacion_temp =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
  df_medicacion_temp["FUENTE"] = fuente
  CambiarNombre(df_medicacion_temp, mapa_nombres)
  df_medicacion = pd.concat([df_medicacion, df_medicacion_temp], ignore_index=True)
  print(df_medicacion_temp.shape)
print(df_medicacion.shape)
df_medicacion

(60460, 10)
(75632, 10)
(115649, 10)
(251741, 10)


,ID_PACIENTE,FARMACO_NOMBRE_COMERCIAL,DOSIS_MEDIA_DIARIA,INICIO_TRATAMIENTO,FIN_TRATAMIENTO,ATC5_NOMBRE,ID_ATC5,ATC7_NOMBRE,ID_ATC7,FUENTE
0,1,MORFINA amp 1% 1 mL (10 mg/1 mL),5,10/04/2020,10/04/2020,ALCALOIDES NATURALES DEL OPIO,N02AA,MORFINA,N02AA01,F_24_04_2020
1,1,DOLQUINE comp 200 mg,2,06/04/2020,10/04/2020,AMINOQUINOLINAS,P01BA,HIDROXICLOROQUINA,P01BA02,F_24_04_2020
2,1,PARACETAMOL comp 1 g,3,07/04/2020,10/04/2020,ANILIDAS,N02BE,PARACETAMOL,N02BE01,F_24_04_2020
3,1,KALETRA/ALUVIA comp (200+50) mg,4,07/04/2020,07/04/2020,ANTIVIRALES PARA EL TRATAMIENTO DE INFECCIONES...,J05AR,RITONAVIR Y LOPINAVIR,J05AE20,F_24_04_2020
4,1,DESINCLOR JABONOSO sol 4% F/500 mL,2,06/04/2020,06/04/2020,BIGUANIDAS Y AMIDINAS,D08AC,CLORHEXIDINA,D08AC02,F_24_04_2020
...,...,...,...,...,...,...,...,...,...,...
251736,2788,SUERO FISIOLOGICO LAV. 100 mL,1,2020-04-02,2020-04-02,SOLUCIONES ELECTROLÍTICAS,B05XA,CLORURO DE SODIO,B05XA03,F_19_04_2021
251737,2789,AZITROMICINA comp 500 mg,1,2020-03-24,2020-03-27,MACRÓLIDOS,J01FA,AZITROMICINA,J01FA10,F_19_04_2021
251738,2789,OXIGENO PLANTA,22,2020-03-23,2020-04-03,GASES MEDICINALES,V03AN,OXÍGENO,V03AN01,F_19_04_2021
251739,2789,SUERO ClNa 100 mL I.V.,2,2020-03-23,2020-04-03,SOLUCIONES QUE AFECTAN EL BALANCE ELECTROLÍTICO,B05BB,ELECTROLITOS,B05BB01,F_19_04_2021


#### Exportación

In [ ]:
df_medicacion.to_csv("Unificados/02_medicacion.csv", index=False, sep=";",encoding='utf-8')

### EDA

In [ ]:
EDA(df_medicacion, "Medicación")

## 3. Constantes vitales.

### LOAD

#### Unificación

In [ ]:
mapa_nombres = {
    'ID_PACIENTE': ['PATIENT ID', 'IDINGRESO'],
    'FECHA_CONSTANTE_INGRESO': ['CONSTANTS_ING/INPAT_FECHA/DATE', 'CONSTANTS_ING_DATE'],
    'HORA_CONSTANTE_INGRESO': ['CONSTANTS_ING/INPAT_HORA/TIME', 'CONSTANTS_ING_TIME'],
    'FC_INGRESO': ['FC/HR_ING/INPAT', 'FC_HR_ING'],
    'GLU_INGRESO': ['GLU/GLY_ING/INPAT', 'GLU_GLY_ING'],
    'SAT_02_INGRESO': ['SAT_02_ING/INPAT', 'SAT_02_ING'],
    'TA_MAX_INGRESO': ['TA_MAX_ING/INPAT', 'TA_MAX_ING'],
    'TA_MIN_INGRESO': ['TA_MIN_ING/INPAT', 'TA_MIN_ING'],
    'TEMPERATURA_INGRESO': ['TEMP_ING/INPAT', 'TEMP_ING'],
    'SAT_02_INGRESO_OBSERVACIONES': ['SAT_02_ING_OBS']
}

urls = {"03.csv":"F_24_04_2020","CDSL_03.csv":"F_20_07_2020","COVID_DSL_02.CSV":"F_19_04_2021"}

df_constantes = pd.DataFrame()
for url, fuente in urls.items():
  df_constantes_temp =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
  df_constantes_temp["FUENTE"] = fuente
  CambiarNombre(df_constantes_temp, mapa_nombres)
  df_constantes = pd.concat([df_constantes, df_constantes_temp], ignore_index=True)
  print(df_constantes_temp.shape)
print(df_constantes.shape)
df_constantes

(55515, 10)
(128005, 11)


<ipython-input-16-642b0952d79c>:18: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_constantes_temp =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))


(224146, 11)
(407666, 11)


,ID_PACIENTE,FECHA_CONSTANTE_INGRESO,HORA_CONSTANTE_INGRESO,FC_INRESO,GLU_INGRESO,SAT_02_INGRESO,TA_MAX_INGRESO,TA_MIN_INGRESO,TEMPERATURA_INGRESO,FUENTE,SAT_02_INGRESO_OBSERVACIONES
0,2316,24/04/2020,00:00,62,0,0,0,0,"36,5",F_24_04_2020,NaN
1,2316,24/04/2020,00:01,69,0,0,0,0,"37,1",F_24_04_2020,NaN
2,2288,24/04/2020,00:04,94,0,0,0,0,"36,8",F_24_04_2020,NaN
3,514,24/04/2020,00:09,0,0,0,0,0,"35,4",F_24_04_2020,NaN
4,181,24/04/2020,00:10,0,0,0,0,0,"36,4",F_24_04_2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...
407661,542,2020-05-17,10:27,0,0,98,0,0,0,F_19_04_2021,SATO2 GN 1L
407662,605,2020-05-17,11:55,0,0,96,0,0,0,F_19_04_2021,NaN
407663,580,2020-11-08,15:46,0,0,35,0,0,0,F_19_04_2021,SATO2 GN 4L
407664,4298,2020-11-08,15:50,0,0,97,0,0,0,F_19_04_2021,SATO2 GN 1L


#### Exportación

In [ ]:
#hay datos que en temperatura vienen con puntos como separador decimal y otros con coma

df_constantes['TEMPERATURA_INGRESO'] = (
    df_constantes['TEMPERATURA_INGRESO']
    .str.replace(',', '.', regex=False)
    .astype(float)
)
df_constantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407666 entries, 0 to 407665
Data columns (total 11 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   ID_PACIENTE                   407666 non-null  int64  
 1   FECHA_CONSTANTE_INGRESO       407666 non-null  object 
 2   HORA_CONSTANTE_INGRESO        407666 non-null  object 
 3   FC_INRESO                     407666 non-null  int64  
 4   GLU_INGRESO                   407666 non-null  int64  
 5   SAT_02_INGRESO                407666 non-null  int64  
 6   TA_MAX_INGRESO                407666 non-null  int64  
 7   TA_MIN_INGRESO                407666 non-null  int64  
 8   TEMPERATURA_INGRESO           186587 non-null  float64
 9   FUENTE                        407666 non-null  object 
 10  SAT_02_INGRESO_OBSERVACIONES  329016 non-null  object 
dtypes: float64(1), int64(6), object(4)
memory usage: 34.2+ MB


In [ ]:
df_constantes.to_csv("Unificados/03_constantes.csv", index=False, sep=";",encoding='utf-8')

### EDA

In [ ]:
EDA(df_constantes, "Constantes Vitales")

## 4. Laboratorio.

### LOAD

#### Unificación

In [ ]:
mapa_nombres = {
    'ID_PACIENTE': ['PATIENT ID', 'IDINGRESO'],
    'LAB_NUMERO': ['PETICION_LABORATORIO/LAB_NUMBER', 'LAB_NUMBER'],
    'LAB_FECHA': ['FECHA_PETICION/LAB_DATE', 'LAB_DATE'],
    'LAB_HORA': ['HORA_PETICION/TIME_LAB', 'TIME_LAB'],
    'LAB_ITEM': ['DETERMINACION/ITEM_LAB', 'ITEM_LAB'],
    'RESULTADO_VALOR': ['RESULTADO/VAL_RESULT', 'VAL_RESULT'],
    'RESULTADO_UNIDAD': ['UNIDADES/UD_RESULT', 'UD_RESULT'],
    'RESULTADO_VALORES_REFERENCIALES': ['VALORES_REFERENCIA/REF_VALUES', 'REF_VALUES']
}

urls = {"04.csv":"F_24_04_2020","CDSL_04.csv":"F_20_07_2020","COVID_DSL_06_v2.CSV":"F_19_04_2021"}

df_laboratorio = pd.DataFrame()
for url, fuente in urls.items():
  df_laboratorio_temp =  CorreccionSaltoLinea(url) if url == "04.csv" else pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
  df_laboratorio_temp["FUENTE"] = fuente
  CambiarNombre(df_laboratorio_temp, mapa_nombres)
  df_laboratorio = pd.concat([df_laboratorio, df_laboratorio_temp], ignore_index=True)
  print(df_laboratorio_temp.shape)
print(df_laboratorio.shape)
df_laboratorio

(396055, 9)
(584043, 9)
(786984, 9)
(1767082, 9)


,ID_PACIENTE,LAB_NUMERO,LAB_FECHA,LAB_HORA,LAB_ITEM,RESULTADO_VALOR,RESULTADO_UNIDAD,RESULTADO_VALORES_REFERENCIALES,FUENTE
0,1,I-1,08-04-2020,00:00,GOT -- GOT (AST),215.0,U/L,<41,F_24_04_2020
1,1,I-1,08-04-2020,00:00,LIN% -- Linfocitos %,3.0,%,(20.0 - 50.0),F_24_04_2020
2,1,I-1,08-04-2020,00:00,MONO -- Monocitos,0.12,x10e3/µL,(0.1 - 0.8),F_24_04_2020
3,1,I-2,07-04-2020,00:00,BAS -- Basófilos,0.02,x10e3/µL,(0.0 - 0.3),F_24_04_2020
4,1,I-2,07-04-2020,00:00,BT -- BILIRRUBINA TOTAL ...,0.43,mg/dL,<1.2,F_24_04_2020
...,...,...,...,...,...,...,...,...,...
1767077,4474,I-1,04-12-2020,00:00,G-CORONAV (RT-PCR) -- Tipo de muestra: EXUDADO,No se detecta,NaN,NaN,F_19_04_2021
1767078,4474,I-2,10-12-2020,00:00,G-CORONAV (RT-PCR) -- Tipo de muestra: EXUDADO,Se detecta,NaN,NaN,F_19_04_2021
1767079,4475,I-1,10-12-2020,00:00,G-CORONAV (RT-PCR) -- Tipo de muestra: EXUDADO,No se detecta,NaN,NaN,F_19_04_2021
1767080,4478,I-1,04-12-2020,00:00,G-CORONAV (RT-PCR) -- Tipo de muestra: EXUDADO,Se detecta,NaN,NaN,F_19_04_2021


#### Exportación

In [ ]:
df_laboratorio.to_csv("Unificados/04_laboratorio.csv", index=False, sep=";",encoding='utf-8')

### EDA

In [ ]:
EDA(df_laboratorio, "Laboratorios")

## 5. CIE10 Urgencia

### LOAD

#### Unificación

In [ ]:
mapa_nombres = {
    'ID_PACIENTE': ['PATIENT ID', 'IDINGRESO']
}

urls = {"05.csv":"F_24_04_2020","CDSL_05.csv":"F_20_07_2020","COVID_DSL_03.CSV":"F_19_04_2021"}

df_cie10_urgencia = pd.DataFrame()
for url, fuente in urls.items():
  df_cie10_urgencia_temp =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
  df_cie10_urgencia_temp["FUENTE"] = fuente
  CambiarNombre(df_cie10_urgencia_temp, mapa_nombres)
  df_cie10_urgencia = pd.concat([df_cie10_urgencia, df_cie10_urgencia_temp], ignore_index=True)
  print(df_cie10_urgencia_temp.shape)
print(df_cie10_urgencia.shape)
df_cie10_urgencia

(1987, 19)
(2454, 19)
(4197, 19)
(8638, 19)


,ID_PACIENTE,DIA_PPAL,DIA_02,DIA_03,DIA_04,DIA_05,DIA_06,DIA_07,DIA_08,DIA_09,DIA_10,DIA_11,DIA_12,PROC_01,PROC_02,PROC_03,PROC_04,PROC_05,FUENTE
0,47,J12.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
1,48,J98.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
2,49,I50.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
3,50,J98.4,N39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
4,51,J18.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8633,2758,E86.0,E87.1,J12.89,J84.9,R68.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,0BJ0XZZ,NaN,NaN,NaN,F_19_04_2021
8634,2759,J96.90,R68.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,NaN,NaN,NaN,NaN,F_19_04_2021
8635,2760,R06.00,J12.89,B97.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,NaN,NaN,NaN,NaN,F_19_04_2021
8636,2761,R68.89,J96.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BW03ZZZ,NaN,NaN,NaN,NaN,F_19_04_2021


#### Exportación

In [ ]:
df_cie10_urgencia.to_csv("Unificados/05_cie10_urgencia.csv", index=False, sep=";",encoding='utf-8')

### EDA

In [ ]:
EDA(df_cie10_urgencia, "CIE10 Urgencia")

## 6. CIE10 Ingreso

### LOAD

#### Unificación

In [ ]:
mapa_nombres = {
    'ID_PACIENTE': ['PATIENT ID', 'IDINGRESO']
}

urls = {"06.csv":"F_24_04_2020","CDSL_06.csv":"F_20_07_2020","COVID_DSL_05.CSV":"F_19_04_2021"}

df_cie10_Ingreso = pd.DataFrame()
for url, fuente in urls.items():
  df_cie10_Ingreso_temp =  pd.read_csv(url,delimiter=DetectaSeparador(url),encoding=DetectaEncoding(url))
  df_cie10_Ingreso_temp["FUENTE"] = fuente
  CambiarNombre(df_cie10_Ingreso_temp, mapa_nombres)
  df_cie10_Ingreso = pd.concat([df_cie10_Ingreso, df_cie10_Ingreso_temp], ignore_index=True)
  print(df_cie10_Ingreso_temp.shape)
print(df_cie10_Ingreso.shape)
df_cie10_Ingreso

(1775, 66)
(2523, 66)
(4427, 66)
(8725, 66)


,ID_PACIENTE,DIA_PPAL,DIA_02,DIA_03,DIA_04,DIA_05,DIA_06,DIA_08,DIA_07,DIA_09,...,PROC_12,PROC_13,PROC_14,PROC_15,PROC_16,PROC_17,PROC_18,PROC_19,PROC_20,FUENTE
0,2293,J18.9,Z20.828,R68.89,R09.02,D80.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
1,2287,J12.89,B97.29,M19.90,F32.9,G25.81,M51.86,Z96.1,Z98.49,Z60.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
2,2269,J18.9,R68.89,J96.90,I10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
3,2265,J12.89,B97.29,J96.00,J84.89,Z99.89,Z95.0,NaN,Z79.52,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
4,2264,J47.0,J12.89,B97.29,J47.1,I10,E11.9,Z87.891,K44.9,Z98.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_24_04_2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8720,2904,R05,Z20.828,R68.89,K29.50,Z98.52,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021
8721,2912,J98.8,R68.89,Z20.828,R79.89,L50.3,Z86.19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021
8722,2916,G44.89,R68.89,Z20.828,R78.81,B95.7,M93.88,M47.812,I35.0,F33.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021
8723,2956,J18.9,Z20.828,I48.91,J90,I11.0,I50.9,H81.399,E11.9,H91.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F_19_04_2021


#### Exportación

In [ ]:
df_cie10_Ingreso.to_csv("Unificados/06_cie10_ingreso.csv", index=False, sep=";",encoding='utf-8')

### EDA

In [ ]:
EDA(df_cie10_Ingreso, "CIE10 Ingreso")


In [ ]:
url = "05.csv";
df_cie10_urgencia =  pd.read_csv(url,delimiter=";",encoding=DetectaEncoding(url))
df_cie10_urgencia

,PATIENT ID,DIA_PPAL,DIA_02,DIA_03,DIA_04,DIA_05,DIA_06,DIA_07,DIA_08,DIA_09,DIA_10,DIA_11,DIA_12,PROC_01,PROC_02,PROC_03,PROC_04,PROC_05
0,47,J12.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48,J98.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49,I50.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50,J98.4,N39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51,J18.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982,2317,J96.92,D64.9,J98.8,D69.6,N19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1983,2318,J84.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,2319,J84.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1985,2320,K20.9,R13.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 7. CIE10 Códigos Diagnosticos

### LOAD

In [ ]:
url = "icd10cm_order_2021.txt";

'''
separador = detectar_separador(url)
print(f"Separador detectado: {repr(separador)}")
'''

#Es un archivo de ancho fijo


# Definir anchos de columna
anchos = [5, 8, 2, 61, 230]  # Ajustar si es necesario
columnas = ['ID', 'CODIGO', 'NIVEL', 'DESCRIPCION_CORTA', 'DESCRIPCION_LARGA'] # Definir encabezados

df_cie10_cod_diag =  pd.read_fwf(url,encoding=DetectaEncoding(url),names=columnas,widths=anchos)
df_cie10_cod_diag

,ID,CODIGO,NIVEL,DESCRIPCION_CORTA,DESCRIPCION_LARGA
0,1,A00,0,Cholera,Cholera
1,2,A000,1,"Cholera due to Vibrio cholerae 01, biovar chol...","Cholera due to Vibrio cholerae 01, biovar chol..."
2,3,A001,1,"Cholera due to Vibrio cholerae 01, biovar eltor","Cholera due to Vibrio cholerae 01, biovar eltor"
3,4,A009,1,"Cholera, unspecified","Cholera, unspecified"
4,5,A01,0,Typhoid and paratyphoid fevers,Typhoid and paratyphoid fevers
...,...,...,...,...,...
95353,95354,Z9981,1,Dependence on supplemental oxygen,Dependence on supplemental oxygen
95354,95355,Z9989,1,Dependence on other enabling machines and devices,Dependence on other enabling machines and devices
95355,95356,U07,0,Emergency use of U07,Emergency use of U07
95356,95357,U070,1,Vaping-related disorder,Vaping-related disorder


#### Exportación

In [ ]:
df_cie10_cod_diag.to_csv("Unificados/07_cie10_cod_diag.csv", index=False, sep=";",encoding='utf-8')

### EDA

In [ ]:
EDA(df_cie10_cod_diag, "CIE10 Códigos Diagnosticos")